In [5]:
!git clone https://@github.com/Web-Team-IITI-Gymkhana/data-bot.git

fatal: destination path 'data-bot' already exists and is not an empty directory.


# Basic Libraries Import

---



In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(os.curdir))
from sklearn import preprocessing
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier

['.config', 'data-bot', 'predicted_company_scores_150.csv', 'sample_data']


# Data Import

---

Companies and their CIK

In [7]:
com= pd.read_csv('/content/data-bot/csv/GoodCom.csv')

com.head(3)

,CIK,CompanyName
0,1459417,2U INC
1,1023731,8X8 INC
2,1580808,A10 NETWORKS INC




---

#Analysis: Finding Threshold To Classify Investability as Good / Neutral / Bad 

>> Regression Analysis Results

In [8]:
predscore= pd.read_csv('/content/predicted_company_scores_150.csv')
predscore.head()

,cik_date,GrossProfit,GrossMargin,WorkingCapitalRatio,EarningPerShare,DebtToEquityRatio,PEratio,ReturnOfEquity,EBIDTAratio,EvRatio,...,growth_rate_label,profitm_label,grossm_label,ro40_label,churnrate_label,EVbyEbidta_label,marketCap_label,magicNum_label,aggregate,predicted_aggregate
0,1262039_2020,2024.40,0.78,1.50,3.01,1.52,49.41,40.47,1101.80,7935446017.82,...,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.38,0.38
1,1262039_2021,2559.20,0.77,1.55,3.75,1.85,95.95,48.38,1433.40,7935446017.82,...,0.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,0.69,0.69
2,944480_2018,23604000.00,0.25,1.57,0.00,0.29,inf,0.00,70959000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.15,0.15
3,944480_2019,20298000.00,0.24,0.89,0.00,0.43,inf,0.00,55258000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,0.08
4,944480_2020,14785000.00,0.26,0.87,0.00,0.27,inf,0.00,33297000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,0.10


>>Manually Finding Accuracy of Regression Analysis using XGBRegressor

In [9]:
diff=abs(predscore['predicted_aggregate']-predscore['aggregate'])
count=0
for x in diff:
  if x<=0.1 and x>=0:
    count+=1
print((count/len(diff))*100)    

97.19101123595506


>>Function to Extract CIK from CIK_Year Format

In [10]:
def cik_scrap(st):
  st=str(st)
  return float(st.split('_')[0])

In [11]:
predscore['CIK']= predscore['cik_date'].apply(cik_scrap)
predscore.head()

,cik_date,GrossProfit,GrossMargin,WorkingCapitalRatio,EarningPerShare,DebtToEquityRatio,PEratio,ReturnOfEquity,EBIDTAratio,EvRatio,...,profitm_label,grossm_label,ro40_label,churnrate_label,EVbyEbidta_label,marketCap_label,magicNum_label,aggregate,predicted_aggregate,CIK
0,1262039_2020,2024.40,0.78,1.50,3.01,1.52,49.41,40.47,1101.80,7935446017.82,...,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.38,0.38,1262039.00
1,1262039_2021,2559.20,0.77,1.55,3.75,1.85,95.95,48.38,1433.40,7935446017.82,...,1.00,1.00,1.00,1.00,0.00,0.00,1.00,0.69,0.69,1262039.00
2,944480_2018,23604000.00,0.25,1.57,0.00,0.29,inf,0.00,70959000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.15,0.15,944480.00
3,944480_2019,20298000.00,0.24,0.89,0.00,0.43,inf,0.00,55258000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,0.08,944480.00
4,944480_2020,14785000.00,0.26,0.87,0.00,0.27,inf,0.00,33297000.00,7935446017.82,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.08,0.10,944480.00


>>**Creating DataFrame 'CIK'  'aggregate' & 'predicted_aggregate' features**

In [12]:
fans= predscore[['CIK','aggregate','predicted_aggregate']]
fans.head()

,CIK,aggregate,predicted_aggregate
0,1262039.00,0.38,0.38
1,1262039.00,0.69,0.69
2,944480.00,0.15,0.15
3,944480.00,0.08,0.08
4,944480.00,0.08,0.10


>> **Merging Companies List with Test Data**

In [13]:
final=pd.merge(fans, com, on='CIK')
final=pd.DataFrame(final.to_numpy(),columns=final.keys())
final.head()

,CIK,aggregate,predicted_aggregate,CompanyName
0,1262039.00,0.38,0.38,FORTINET INC
1,1262039.00,0.69,0.69,FORTINET INC
2,944480.00,0.15,0.15,GSE SYSTEMS INC
3,944480.00,0.08,0.08,GSE SYSTEMS INC
4,944480.00,0.08,0.10,GSE SYSTEMS INC


>> Mapping CIK number with Company Name

In [14]:
cik_com_map=dict(zip(final.CIK,final.CompanyName))
#cik_com_map

>> **Grouping Companies to Get Average Performance **

In [15]:
final2=final.groupby(['CompanyName']).mean()
final2.head()

,CIK,aggregate,predicted_aggregate
CompanyName,,,
FORTINET INC,1262039.00,0.54,0.54
GSE SYSTEMS INC,944480.00,0.10,0.11
GUIDEWIRE SOFTWARE INC,1528396.00,0.13,0.16
HOPTO INC,1021435.00,0.26,0.28
HUBSPOT INC,1404655.00,0.28,0.28


>>*Scores Achieved by companies out of 100*

In [16]:
def multiplier(n):
  return n*100

final2['aggregate']=  final2['aggregate'].apply(multiplier)
final2['predicted_aggregate']= final2['predicted_aggregate'].apply(multiplier)

In [17]:
final2.head()

,CIK,aggregate,predicted_aggregate
CompanyName,,,
FORTINET INC,1262039.00,53.85,53.84
GSE SYSTEMS INC,944480.00,10.26,10.97
GUIDEWIRE SOFTWARE INC,1528396.00,12.82,15.60
HOPTO INC,1021435.00,25.64,27.85
HUBSPOT INC,1404655.00,28.21,28.22


# Statistical Distribution of Score to Calculate Threshold

In [18]:
final2.describe()

,CIK,aggregate,predicted_aggregate
count,67.00,67.00,67.00
mean,1155709.36,28.08,28.34
std,337484.63,15.10,14.60
min,50471.00,0.00,0.15
25%,919281.50,15.38,16.38
50%,1102993.00,26.92,26.94
75%,1416985.00,37.18,35.56
max,1725579.00,64.10,63.62


# Classification of Companies 

---



In [19]:
def bng(n):
  if n<=15:
    return 'Bad'
  elif n>15 and n<=35:
    return 'Neutral'
  else: 
    return 'Good'

final2['Investability']= final2['aggregate'].apply(bng)

In [20]:
final2.head(10)

,CIK,aggregate,predicted_aggregate,Investability
CompanyName,,,,
FORTINET INC,1262039.00,53.85,53.84,Good
GSE SYSTEMS INC,944480.00,10.26,10.97,Bad
GUIDEWIRE SOFTWARE INC,1528396.00,12.82,15.60,Bad
HOPTO INC,1021435.00,25.64,27.85,Neutral
HUBSPOT INC,1404655.00,28.21,28.22,Neutral
IMAGEWARE SYSTEMS INC,941685.00,10.26,10.25,Bad
INTELLIGENT SYSTEM CORP,320340.00,32.69,32.32,Neutral
INTELLINETICS INC,1081745.00,10.26,10.27,Bad
INTRUSION INC,736012.00,30.77,33.02,Neutral


In [21]:
final2.to_csv('Investability.csv')